# Курсовая работа по СиАОДу
## Выполнил студент группы Глушков Игнатий Игоревич БФИ2202

In [1]:
import sys
import csv
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QFileDialog, QMessageBox, QVBoxLayout, QWidget, QSpacerItem, QSizePolicy, QLineEdit, QHBoxLayout
import matplotlib.pyplot as plt

In [2]:
class HashTable:
    def __init__(self):
        self.capacity = 100
        self.size = 0
        self.threshold = 0.75
        self.data = [None] * self.capacity
    
    def _hash(self, key):
        return hash(key) % self.capacity
    
    def _resize(self):
        new_capacity = self.capacity * 2
        new_data = [None] * new_capacity
        for item in self.data:
            if item is not None:
                index = self._hash(item[0])
                new_data[index] = item
        self.data = new_data
        self.capacity = new_capacity
    
    def set(self, key, value):
        if self.size / self.capacity >= self.threshold:
            self._resize()
        index = self._hash(key)
        self.data[index] = (key, value)
        self.size += 1
    
    def get(self, key):
        index = self._hash(key)
        return self.data[index][1] if self.data[index] is not None else None
    
    def increment(self, key, value):
        if self.size / self.capacity >= self.threshold:
            self._resize()
        index = self._hash(key)
        current_value = self.data[index][1] if self.data[index] is not None else 0
        self.data[index] = (key, current_value + value)
        self.size += 1
    
    def items(self):
        return [(item[0], item[1]) for item in self.data if item is not None]
    
    def clear(self):
        self.data = [None] * self.capacity
        self.size = 0


In [3]:
def showPie(data):    
    total_revenue = sum(float(row['Общая стоимость']) for row in data)
    revenue_percentage = {}
    for row in data:
        product_name = row['Название товара']
        total_price = float(row['Общая стоимость'])
        revenue_percentage[product_name] = (total_price / total_revenue) * 100

    labels = revenue_percentage.keys()
    sizes = revenue_percentage.values()

    plt.figure(figsize=(10, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Процент выручки каждого товара')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

In [4]:
def insertion_sort(items, key=lambda x: x):
    for i in range(1, len(items)):
        current_item = items[i]
        j = i - 1
        while j >= 0 and key(items[j]) < key(current_item):
            items[j + 1] = items[j]
            j -= 1
        items[j + 1] = current_item
    return items

In [5]:

class SalesAnalyzer(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Sales Analyzer")
        self.setGeometry(100, 100, 600, 400)
        
        self.open_button = QPushButton("Открыть CSV файл", self)
        self.open_button.clicked.connect(self.open_file)
        
        self.calculate_button = QPushButton("Сформировать отчет", self)
        self.calculate_button.clicked.connect(self.save_report)
        self.calculate_button.setEnabled(False)

        self.search_label = QLabel("Введите название товара для поиска:", self)
        self.search_input = QLineEdit(self)
        self.search_button = QPushButton("Поиск", self)
        self.search_button.clicked.connect(self.search_product)
        self.search_button.setEnabled(False)
        
        self.status_label = QLabel("Выберите CSV файл для анализа", self)
        self.revenue_label = QLabel("", self)
        self.top_selling_label = QLabel("", self)
        self.top_revenue_label = QLabel("", self)
        self.search_result_label = QLabel("", self)
        
        layout = QVBoxLayout()
        layout.addWidget(self.open_button)
        layout.addWidget(self.calculate_button)
        
        search_layout = QHBoxLayout()
        search_layout.addWidget(self.search_label)
        search_layout.addWidget(self.search_input)
        search_layout.addWidget(self.search_button)
        
        layout.addLayout(search_layout)
        
        layout.addWidget(self.status_label)
        layout.addWidget(self.revenue_label)
        layout.addWidget(self.top_selling_label)
        layout.addWidget(self.top_revenue_label)
        layout.addWidget(self.search_result_label)
        
        layout.addSpacerItem(QSpacerItem(20, 40, QSizePolicy.Minimum, QSizePolicy.Expanding))
        
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        
        self.file_path = None
        self.data = None
        self.total_revenue = 0.0
        self.sales_count = HashTable()
        self.revenue_by_product = HashTable()
        self.top_selling_product = None
        self.top_revenue_product = None
        
    def open_file(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Открыть CSV файл", "", "CSV Files (*.csv);;All Files (*)", options=options)
        if file_path:
            self.file_path = file_path
            self.data = self.read_csv(file_path)
            if self.data and self.calculate_statistics(self.data):
                self.display_statistics()
                self.status_label.setText("Файл загружен и данные рассчитаны.")
                self.calculate_button.setEnabled(True)
                self.search_button.setEnabled(True)
            else:
                self.status_label.setText("Ошибка при загрузке файла.")
                self.calculate_button.setEnabled(False)
                self.search_button.setEnabled(False)

    def read_csv(self, file_path):
        try:
            with open(file_path, newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                data = [row for row in reader]
                if not data:
                    raise ValueError("Файл пуст или неверно составлен")
                
                self.data = data
                return data
        except (FileNotFoundError, ValueError, TypeError, Exception) as e:
            QMessageBox.critical(self, "Ошибка", f"Ошибка при чтении файла: {e}")
            return None
        
    def calculate_statistics(self, data):
        self.total_revenue = 0.0
        self.sales_count.clear()
        self.revenue_by_product.clear()

        try:
            for row in data:
                product_name = row['Название товара']
                count = int(row['Количество продаж'])
                total_price = float(row['Общая стоимость'])
                
                self.total_revenue += total_price
                self.sales_count.increment(product_name, count)
                self.revenue_by_product.increment(product_name, total_price)

            self.top_selling_product = max(self.sales_count.items(), key=lambda x: x[1])[0]
            self.top_revenue_product = max(self.revenue_by_product.items(), key=lambda x: x[1])[0]
            return True
        except KeyError as ke:
            QMessageBox.critical(self, "Ошибка", f"Отсутствует ожидаемый столбец в файле: {ke}")
            return False
        except ValueError as ve:
            QMessageBox.critical(self, "Ошибка", f"Ошибка в формате данных: {ve}")
            return False
        except TypeError as te:
            QMessageBox.critical(self, "Ошибка", f"Ошибка в данных: {te}")
            return False
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Ошибка при обработке данных: {e}")
            return False
    
    def display_statistics(self): 
        showPie(self.data)
        self.revenue_label.setText(f"Общая выручка магазина: {self.total_revenue:.2f} руб.")
        self.top_selling_label.setText(f"Товар с наибольшими продажами: {self.top_selling_product}")
        self.top_revenue_label.setText(f"Товар с наибольшей выручкой: {self.top_revenue_product}")
    
    def save_report(self):
        report = self.create_report(self.total_revenue, self.sales_count, self.revenue_by_product)
        save_path, _ = QFileDialog.getSaveFileName(self, "Сохранить отчет", "", "Text Files (*.txt);;All Files (*)")
        if save_path:
            try:
                with open(save_path, "w", encoding="utf-8") as file:
                    file.write(report)
                QMessageBox.information(self, "Успех", "Отчет успешно сохранен.")
            except Exception as e:
                QMessageBox.critical(self, "Ошибка", f"Ошибка при сохранении отчета: {e}")
    
    def create_report(self, total_revenue, sales_count, revenue_by_product):
        report = []
        report.append(f"Общая выручка магазина: {total_revenue:.2f} руб.\n")
        
        report.append("Количество проданных единиц каждого товара:")
        sorted_sales = insertion_sort(list(sales_count.items()), key=lambda x: x[1])
        for product, count in sorted_sales:
            report.append(f"{product}: {count} шт.")
        
        report.append("\nДоля каждого товара в общей выручке:")
        sorted_revenue = insertion_sort(list(revenue_by_product.items()), key=lambda x: x[1])
        for product, revenue in sorted_revenue:
            share = (revenue / total_revenue) * 100
            report.append(f"{product}: {share:.2f}%")
        
        report.append(f"\nТовар, который был продан наибольшее количество раз: {self.top_selling_product}")
        report.append(f"Товар, который принес наибольшую выручку: {self.top_revenue_product}")
        
        return "\n".join(report)

    def search_product(self):
        product_name = self.search_input.text().strip()
        if not product_name:
            self.search_result_label.setText("Введите название товара для поиска.")
            return
        
        found = False
        for row in self.data:
            if row['Название товара'].lower() == product_name.lower():
                count = int(row['Количество продаж'])
                total_price = float(row['Общая стоимость'])
                self.search_result_label.setText(f"Товар: {row['Название товара']}, Продажи: {count}, Выручка: {total_price:.2f} руб.")
                found = True
                break
        
        if not found:
            self.search_result_label.setText("Товар не найден.")
        

In [ ]:
def main():
    app = QApplication(sys.argv)
    window = SalesAnalyzer()
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()
